In [65]:
import pandas as pd
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

torch.cuda.is_available()

False

In [66]:
df = pd.read_csv('livedoor_news.csv')
label = df['label'].tolist()
train = df['sentence'].tolist()

df.head() #{0 : 'IT', 1 : 'スポーツ', 2 : '映画'}

,Unnamed: 0,label,sentence
0,2068,2,インタビュー：マックG監督「リメイクしたい日本のアニメは『AKIRA』かな」
1,898,1,【Sports Watch】なでしこジャパン勝利、決勝＝アメリカ戦の攻略ポイントは…
2,1971,2,女子高生シンガーのKyleeが「美人映写技師との恋」を描いた映画の主題歌担当に
3,1288,1,銀メダル・三宅宏実、試合後、同じ言葉が32回飛び出した!?
4,322,0,七人の侍によって開発された新素材！ NECのLaVie Zで使われた新素材の開発秘話【デジ通】


In [67]:
train_text, test_text, train_label, test_label = train_test_split(train, label, train_size=0.8)

JP_tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

#トークン化
train_encodings = JP_tokenizer(train_text, truncation= True, padding=True)
test_encodings = JP_tokenizer(test_text, truncation= True, padding= True)

In [68]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_label)
val_dataset = IMDbDataset(test_encodings, test_label)

In [69]:
model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', num_labels=3)

# Linear(in_features=768, out_features=9, bias=True)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

KeyboardInterrupt: 